## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-04-00-13 +0000,nypost,Anxious Venezuelan security forces open fire o...,https://nypost.com/2026/01/05/world-news/anxio...
1,2026-01-06-03-52-53 +0000,nyt,Venezuela Live Updates: At U.N. Emergency Meet...,https://www.nytimes.com/live/2026/01/05/world/...
2,2026-01-06-03-51-57 +0000,wapo,"Corporation for Public Broadcasting, gutted of...",https://www.washingtonpost.com/business/2026/0...
3,2026-01-06-03-49-53 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/05/world/...
4,2026-01-06-03-47-29 +0000,bbc,Venezuela swears in interim president after de...,https://www.bbc.com/news/articles/ce8gen8nnvlo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2419/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,venezuela,68
37,maduro,55
41,trump,50
128,new,21
107,oil,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
138,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,205
287,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,196
290,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,193
174,2026-01-05-19-32-24 +0000,nyt,"Who Is Delcy Rodríguez, Venezuela’s New Interi...",https://www.nytimes.com/2026/01/05/world/ameri...,187
275,2026-01-05-10-47-00 +0000,wsj,Oil Prices Fall as U.S. Capture of Venezuela’s...,https://www.wsj.com/business/energy-oil/asian-...,176


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
138,205,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
25,72,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
299,61,2026-01-05-07-55-46 +0000,nypost,Gov. Tim Walz expected to drop bid for re-elec...,https://nypost.com/2026/01/05/us-news/gov-tim-...
163,52,2026-01-05-20-12-00 +0000,wsj,Oil Rises As Market Digests Venezuelan Preside...,https://www.wsj.com/finance/commodities-future...
237,49,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
37,41,2026-01-06-01-37-29 +0000,nypost,Newsom claims he was ‘physically’ in middle of...,https://nypost.com/2026/01/05/us-news/newsom-b...
159,39,2026-01-05-20-18-05 +0000,nypost,Will Amy Klobuchar run for Minnesota governor?...,https://nypost.com/2026/01/05/us-news/all-eyes...
67,37,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
73,36,2026-01-05-23-38-20 +0000,nypost,Nvidia CEO Jensen Huang calls AI titan’s lates...,https://nypost.com/2026/01/05/business/nvidia-...
239,31,2026-01-05-15-15-57 +0000,nypost,"Sen. Mark Kelly at risk of demotion, cut in mi...",https://nypost.com/2026/01/05/us-news/sen-mark...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
